# Imports

In [1]:
import torch
import torch.nn as nn
from utils.datautils import *
from utils.MLutils import *
from utils.resources import *
from transformers import BertTokenizerFast
from sklearn.model_selection import train_test_split
from transformers import BertModel
from data.variables import *

d:\AA2\torch-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Procesamiento

In [2]:
linux = True
device = None

if linux:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
else:
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

print("usando:", device)

usando: cuda


In [3]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")

## Busqueda de fuentes

- Fuente 1: Conjunto de preguntas en espa;ol
- Fuente 2: Dataset provisto para Notebook 10
- Fuente 3: Dataset sintetico generado con Gemini
- Fuente 4: Articulos de Wikipedia
- Fuente 5: Subtitulos de peliculas
- Fuente 6: Mixture of preguntas y afirmaciones

In [4]:
questions, question_for_mixture = get_questions()
oraciones_rnn = get_notebook_dataset()
oraciones_sinteticas = get_gemini_dataset()
frases_wikipedia = get_wikipedia_dataset()
esperando_la_carroza, frases_relatos_salvajes = get_pelis_dataset()
mixtures = get_mixture_dataset(oraciones_sinteticas, question_for_mixture)

Se descargaron 5000 preguntas en Español.
Se descargaron 997 oraciones en Español (del dataset del notebook 10).
Hay 1413 oraciones sintéticas.
Se cargaron 6648 frases de Wikipedia.
Se extrajeron 947 frases completas y se guardaron en 'dialogos_esperando_la_carroza.json'
Frases extraídas en total: 947
✅ Se extrajeron 1000 frases de Relatos Salvajes.


## Juntamos las fuentes

In [5]:
oraciones_raw = questions + oraciones_rnn + oraciones_sinteticas + frases_wikipedia + esperando_la_carroza  + frases_relatos_salvajes + mixtures

print('Cantidad total de oraciones:',len(oraciones_raw))
print('Cantidad de oraciones de preguntas:',len(questions))
print('Cantidad de oraciones en espa;ol de hugging face:',len(oraciones_rnn))
print('Cantidad de oraciones sintéticas:',len(oraciones_sinteticas))
print('Cantidad de oraciones de Wikipedia:',len(frases_wikipedia))
print('Cantidad de oraciones de Esperando la carroza:',len(esperando_la_carroza))
print('Cantidad de oraciones de Relatos Salvajes:',len(frases_relatos_salvajes))
print('Cantidad de oraciones de mixture:',len(mixtures))

print("Algunas oraciones aleatorias:")
random.sample(oraciones_raw, 5)

Cantidad total de oraciones: 20244
Cantidad de oraciones de preguntas: 5000
Cantidad de oraciones en espa;ol de hugging face: 997
Cantidad de oraciones sintéticas: 1413
Cantidad de oraciones de Wikipedia: 6648
Cantidad de oraciones de Esperando la carroza: 947
Cantidad de oraciones de Relatos Salvajes: 1000
Cantidad de oraciones de mixture: 4239
Algunas oraciones aleatorias:


['Ese año, organizó los territorios ocupados en Italia en lo que se conoció como la República Cisalpina.',
 'De entre ellos cabe destacar a John Locke y Voltaire, cuyas ideas fueron muy importantes para el desarrollo de la noción moderna de derechos.',
 '¿Cuál fue el género de la lista de best seller de The New York Times en el que se clasificó El mundo sin nosotros?',
 'En carrera, se le agregan los segundos al tiempo final que haya obtenido en su última vuelta, con la posible pérdida de posiciones.',
 '¿Qué causó el aumento del déficit fiscal en Uruguay? El Pumpkin Spice Latte de Starbucks es famoso.']

Separamos en conjuntos de `train` y `test` con el tokenizer de `BERT`

In [6]:
train_sents, test_sents = train_test_split(oraciones_raw, test_size=0.05, random_state=42)

dataloader_train = get_dataloader(oraciones_raw=oraciones_raw, max_length=64, batch_size=64, device=device, tokenizer=tokenizer)
dataloader_test = get_dataloader(oraciones_raw=test_sents, max_length=64, batch_size=64, device=device, tokenizer=tokenizer)

print(len(train_sents))
print(len(test_sents))

19231
1013


## Importamos el modelo

In [7]:
from train.RNNBidirectionalAttention import PunctuationCapitalizationRNNBidirectionalAttention

model_name = "bert-base-multilingual-cased"
bert_model = BertModel.from_pretrained(model_name)

for param in bert_model.parameters():
    param.requires_grad = False

N = 2
for layer in bert_model.encoder.layer[-N:]:
    for param in layer.parameters():
        param.requires_grad = True

for param in bert_model.pooler.parameters():
    param.requires_grad = True

### Entrenamiento

In [8]:
model = PunctuationCapitalizationRNNBidirectionalAttention(
    bert_model = bert_model,
    hidden_dim=256,
    num_punct_start_classes=len(PUNCT_START_TAGS),
    num_punct_end_classes=len(PUNCT_END_TAGS),
    num_cap_classes=len(CAP_TAGS)
).to(device)


start_w, end_w, cap_w = compute_class_weights(
    dataloader_train,
    num_classes_list=[
        len(PUNCT_START_TAGS),
        len(PUNCT_END_TAGS),
        len(CAP_TAGS)
    ],
    device=device,
    beta=0.7
)

criterion_start = nn.CrossEntropyLoss(ignore_index=-100, weight=start_w)
criterion_end   = nn.CrossEntropyLoss(ignore_index=-100, weight=end_w)
criterion_cap   = nn.CrossEntropyLoss(ignore_index=-100, weight=cap_w)

trainable_params = [
    p for p in bert_model.parameters() if p.requires_grad
] + list(model.projection.parameters()) \
  + list(model.lstm1.parameters()) \
  + list(model.lstm2.parameters()) \
  + list(model.punct_start_classifier.parameters()) \
  + list(model.punct_end_classifier.parameters()) \
  + list(model.cap_classifier.parameters())

optimizer = torch.optim.AdamW(trainable_params, lr=2e-5)

In [9]:
train(
      model, 
      dataloader_train=dataloader_train, 
      optimizer=optimizer, 
      criterion_start=criterion_start,
      criterion_end=criterion_end, 
      criterion_cap = criterion_cap, 
      device=device, 
      epochs=20
  )

Epoch 1 | Train Loss: 2.2058
Epoch 2 | Train Loss: 1.6434


KeyboardInterrupt: 

### Modelo cargado

In [ ]:
model = PunctuationCapitalizationRNN(
    bert_model = bert_model,
    hidden_dim=256,
    num_punct_start_classes=len(PUNCT_START_TAGS),
    num_punct_end_classes=len(PUNCT_END_TAGS),
    num_cap_classes=len(CAP_TAGS)
).to(device)

model.load_state_dict(torch.load("model_rnn_tres_cabezas.pt", map_location=device))

In [ ]:
entrada = "no te lo puedo creer messi se va a retirar"

print(f"Entrada: {entrada}")
print(f"Salida: {predict_and_reconstruct(model, entrada, tokenizer, device, verbose=False)}")

## Evaluacion

In [ ]:
evaluate(model, dataloader_test, device)

## Prueba de control de Overfitting

In [ ]:
frases = ["¿QUE?"]
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")

train_loader = get_dataloader(frases, max_length=25, batch_size=1, device=device,tokenizer=tokenizer)

model = PunctuationCapitalizationRNN(
    bert_model=bert_model,
    hidden_dim=64,
    num_punct_start_classes=5,
    num_punct_end_classes=5,
    num_cap_classes=4
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)  # Alto LR

criterion_punct_start = nn.CrossEntropyLoss(ignore_index=-100)
criterion_punct_end   = nn.CrossEntropyLoss(ignore_index=-100)
criterion_cap         = nn.CrossEntropyLoss(ignore_index=-100)

train(
    model,
    dataloader_train=train_loader,
    optimizer=optimizer,
    criterion_start=criterion_punct_start,
    criterion_end=criterion_punct_end,
    criterion_cap=criterion_cap,
    device=device,
    epochs=200
)

entrada = "que"
print("Predicción:", predict_and_reconstruct(model, entrada, tokenizer, device))


## Export modelo

In [ ]:
torch.save(model.state_dict(), "model_rnn_tres_cabezas.pt")

## Generacion CSV TP

In [ ]:
from utils.datautils import *

In [ ]:
# Ruta a un archivo TXT con párrafos (una instancia por párrafo)
ruta_txt = "predict/prueba.txt"

# Ejecutar predicciones y guardar CSV
df_predicciones = predicciones_TP(ruta_txt, model, tokenizer, device, max_length=128, verbose=True)

# Mostrar las primeras filas del dataframe con las predicciones
print(df_predicciones)